# Echo-A 1B5 (Memory Finetune)
This continues off from `Echo-A-1B5-basemodel.ipynb` to perform the full memory finetune & testing process

> This project assumes you have the rwkv-infctx conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps

## Optional: Download the pretrained model
(if you want to skip the the basemodel train + instruct tune)


In [ ]:
# Init required dirs
!mkdir -p ../../../model/
!mkdir -p ../../../datapath/
!mkdir -p ../../../checkpoint/

# Download the Stage2.pth file
!rm -rf ../../../model/Echo-A-1B5-Stage2.pth
!cd ../../../model/ && wget https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/Echo-A-1B5-Stage2.pth
!ls -alh ../../../model/Echo-A-1B5-Stage2.pth

## Prepare the dataset

Prepare and preload the finetuning process dataset

In [2]:
%%script bash
# Reset the dataset dir
mkdir -p ./dataset
rm -rf ./dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

# We do a strong bias for smaller word count, to teach the concept from scratch
# so that the model can learn the function
python ./memory_script/gen_limited_segmented_jsonl.py ./dataset/word-2-count.jsonl  2  20000 &
python ./memory_script/gen_limited_segmented_jsonl.py ./dataset/word-5-count.jsonl  5  20000 &
python ./memory_script/gen_limited_segmented_jsonl.py ./dataset/word-10-count.jsonl 10 20000 &
python ./memory_script/gen_limited_segmented_jsonl.py ./dataset/word-20-count.jsonl 20 20000 &
python ./memory_script/gen_limited_segmented_jsonl.py ./dataset/word-40-count.jsonl 40 20000 &
python ./memory_script/gen_limited_segmented_jsonl.py ./dataset/word-80-count.jsonl 80 20000 &

# With a slight mix of the larger word count
python ./memory_script/gen_limited_segmented_jsonl.py ./dataset/word-100-count.jsonl 100 5000 &
python ./memory_script/gen_limited_segmented_jsonl.py ./dataset/word-200-count.jsonl 100 5000 &

wait
echo "## Done ##"

## Generating word reptition dataset ##
Generated JSONL file with - 2 max words, 20000 samples - at ./dataset/word-2-count.jsonl
Generated JSONL file with - 5 max words, 20000 samples - at ./dataset/word-5-count.jsonl
Generated JSONL file with - 10 max words, 20000 samples - at ./dataset/word-10-count.jsonl
Generated JSONL file with - 20 max words, 20000 samples - at ./dataset/word-20-count.jsonl
Generated JSONL file with - 100 max words, 10000 samples - at ./dataset/word-100-count.jsonl
Generated JSONL file with - 40 max words, 20000 samples - at ./dataset/word-40-count.jsonl
Generated JSONL file with - 80 max words, 20000 samples - at ./dataset/word-80-count.jsonl
## Done ##


## Configure your environment settings
(!Important: you will need to rerun the below cell, if you restart your kernel)

In [13]:
DEEPSPEED_STRAT="deepspeed_stage_1"
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="Echo-A-1B5"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v4neo/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

DEEPSPEED_STRAT: deepspeed_stage_2_offload
ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /home/picocreator/rwkv-proj/picocreator-memory-experiment/notebook/experiment/memory
TRAINER_DIR: /home/picocreator/rwkv-proj/picocreator-memory-experiment/RWKV-v4neo
PROJECT_DIR: /home/picocreator/rwkv-proj/picocreator-memory-experiment


## Stage 3 : Simple Memory finetuning

In [4]:
# Lets preload the requried dataset (enwiki_100k)
!cd "{TRAINER_DIR}" && \
    python3 preload_dataset.py "{NOTEBOOK_DIR}/Echo-A-1B5-mem-finetune-1.yaml"

# Ensure the checkpoint directory exists
!cd "{TRAINER_DIR}" && mkdir -p "../checkpoint/Echo-A-1B5-mem-finetune-1/"

Extracting data files: 100%|█████████████████████| 1/1 [00:00<00:00, 322.86it/s]
Dataset json downloaded and prepared to /home/picocreator/.cache/huggingface/datasets/json/default-6bf53b52af0b7239/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 58.81it/s]


In [6]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python new_train.py fit \
        -c "{NOTEBOOK_DIR}/Echo-A-1B5-mem-finetune-1.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Finetune-1 (bs=256, train-ctx=512, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --model.ctx_len=512

[2023-07-11 20:58:08,627] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1'
/home/picocreator/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/fabric/utilities/seed.py:39: UserWarning: No seed found, seed set to 1487066888
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 1487066888
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.5
wandb: Run data is saved locally in ./wandb/run-20230711_205810-aq3rvmce
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run Echo-A-1B5 - Mem-Finetune-1 (bs=256, train-ctx=1024)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-Memory-Experiment
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-Memory-Experiment/runs/aq3rvmce
Using /home/picocreator/.cache/torch_extensions/py311_cu117 as PyTorc

In [14]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint_ds1.py \
        "../checkpoint/Echo-A-1B5-mem-finetune-1/last.ckpt" \
        "{NOTEBOOK_DIR}/Echo-A-1B5-mem-finetune-1.yaml"

[2023-07-12 16:26:02,953] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1'
/home/picocreator/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/fabric/connector.py:555: UserWarning: bf16 is supported for historical reasons but its usage is discouraged. Please set your precision to bf16-mixed instead!
  rank_zero_warn(
Traceback (most recent call last):
  File "/home/picocreator/rwkv-proj/picocreator-memory-experiment/RWKV-v4neo/export_checkpoint_ds1.py", line 37, in <module>
    trainer = RWKVLightningTrainer(**trainer_config)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/picocreator/rwkv-proj/picocreator-memory-experiment/RWKV-v4neo/src/trainer.py", line 84, in __init__
    super().__init__(*args, **kwargs)
  File "/home/picocreator/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/utilities/argparse.py", line 69,

In [11]:
# Lets move and save this model
!cd "{TRAINER_DIR}" && cp ../checkpoint/Echo-A-1B5-mem-finetune/last.ckpt/rwkv_model.pth ../model/Echo-A-1B5-Tune1.pth
!cd "{TRAINER_DIR}" && ls -alh ../model/Echo-A-1B5-Tune1.pth

-rw-rw-r-- 1 picocreator picocreator 5.7G Jul 12 13:49 ../model/Echo-A-1B5-Tune1.pth


In [12]:
# Lets do a quick dragon prompt validation
!python3 ./memory_script/eval_memory_guided.py "{PROJECT_DIR}/model/Echo-A-1B5-Tune1.pth"

Using /home/picocreator/.cache/torch_extensions/py311_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/picocreator/.cache/torch_extensions/py311_cu117/wkv_cuda/build.ninja...
Building extension module wkv_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module wkv_cuda...
RWKV_JIT_ON 1 RWKV_CUDA_ON 1 RESCALE_LAYER 0

Loading /home/picocreator/rwkv-proj/picocreator-memory-experiment/model/Echo-A-1B5-Tune1.pth ...
Strategy: (total 24+1=25 layers)
* cuda [float32, float32], store 25 layers
0-cuda-float32-float32 1-cuda-float32-float32 2-cuda-float32-float32 3-cuda-float32-float32 4-cuda-float32-float32 5-cuda-float32-float32 6-cuda-float32-float32 7-cuda-float32-float32 8-cuda-float32-float32 9-cuda-float32-float32 10-cuda-float32-float32 11-cuda-float32-float32 12-cuda-float32-float32 13-cuda-float32-float32 14-cuda-floa